In [0]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:

dataset_dir = 'drive/Plant Classification Using C-CNN/dataset'  #Change Directory here for dataset   

In [0]:
SPECIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen',
              'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse',
              'Small-flowered Cranesbill', 'Sugar beet']

In [0]:
for species in SPECIES:
    print('{} {} images'.format(species, len(os.listdir(os.path.join(dataset_dir, species)))))

Black-grass 310 images
Charlock 452 images
Cleavers 335 images
Common Chickweed 713 images
Common wheat 253 images
Fat Hen 538 images
Loose Silky-bent 766 images
Maize 257 images
Scentless Mayweed 607 images
Shepherds Purse 274 images
Small-flowered Cranesbill 586 images
Sugar beet 463 images


In [0]:
train = []

for spec_num, species in enumerate(SPECIES):
    for file in os.listdir(os.path.join(dataset_dir, species)):
        train.append(['drive/Plant Classification Using C-CNN/dataset/{}/{}'.format(species, file), spec_num, species]) #Change Directory here for  drive/Plant Classification Using C-CNN/dataset
        
train = pd.DataFrame(train, columns=['file', 'spec_num', 'species'])

print('Training Data: ',train.shape)

Training Data:  (5554, 3)


In [0]:
print(train.head())

                                                file  spec_num      species
0  drive/Plant Classification Using C-CNN/dataset...         0  Black-grass
1  drive/Plant Classification Using C-CNN/dataset...         0  Black-grass
2  drive/Plant Classification Using C-CNN/dataset...         0  Black-grass
3  drive/Plant Classification Using C-CNN/dataset...         0  Black-grass
4  drive/Plant Classification Using C-CNN/dataset...         0  Black-grass


In [0]:
def Masking(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower= np.array([60 - sensitivity, 100, 50])
    upper= np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(hsv_image, lower, upper)
    Morph = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, Morph)
    
    return mask

def Segmenting(img):
    mask = Masking(img)
    segmented = cv2.bitwise_and(img, img, mask = mask)
    return segmented

def Sharpening(img):
    img_blur = cv2.GaussianBlur(img, (0, 0), 3)
    sharp = cv2.addWeighted(img, 1.5, img_blur, -0.5, 0)
    return sharp

In [0]:
%%time

x_train = []

for i in range(len(train)):
    img = cv2.imread(train['file'][i])
    img = cv2.resize(img,dsize=(256,256))
    img_seg = Segmenting(img)
    img_sharp = Sharpening(img_seg)
    img_rgb = cv2.cvtColor( img_sharp, cv2.COLOR_RGB2GRAY )
    img_grey = np.reshape(img_rgb,(256,256,1))
    x_train.append(np.concatenate((np.array(img),np.array(img_grey)),axis=2))
    if ((i % 500) == 0):
      print(i,' images has been processed.')

x_train = np.array(x_train)

0  images has been processed.
500  images has been processed.
1000  images has been processed.
1500  images has been processed.
2000  images has been processed.
2500  images has been processed.
3000  images has been processed.
3500  images has been processed.
4000  images has been processed.
4500  images has been processed.
5000  images has been processed.
5500  images has been processed.
CPU times: user 1min, sys: 4.18 s, total: 1min 4s
Wall time: 44min 50s


In [0]:
from scipy.io import savemat
data ={}
data["train"] = x_train
savemat("drive/Plant Classification Using C-CNN/train/Image_Processed_data.mat", data) #Change directory here to save .mat dataset



In [0]:
from keras.utils.np_utils import to_categorical
labels = train['spec_num']
labels = to_categorical(labels, num_classes = len(SPECIES))

Using TensorFlow backend.


In [0]:
%%time
from scipy.io import savemat
data ={}
data["train"] = x_train
data["train_labels"] = labels
savemat("drive/Plant Classification Using C-CNN/train/Image_Processed_1data.mat", data)#Change directory here to save .mat dataset

CPU times: user 1min 11s, sys: 664 ms, total: 1min 12s
Wall time: 4min 24s
